# Testing Loading model and running predictions with an appropriate output

In [107]:
def remove_set(a,b):
    return list(set(a)-set(b))

In [108]:
import joblib
model = joblib.load('./dota_picks_trained.h5')

In [110]:
hero_json_file = os.path.join(".","data","heroes.json")
import json
hero_ids = []
with open(hero_json_file) as json_file:
    data = json.load(json_file)
    for i in range(len(data)):
        currDict = data[i]
        hero_ids.append(currDict['id'])
matchups_all = []
for i in hero_ids:
    for j in hero_ids:
        if i != j:
            matchups_all.append(str(i) + "-" + str(j))

In [111]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
ohe = OneHotEncoder()
X_all = np.array(matchups_all).reshape(-1,1)
ohe.fit(X_all)

OneHotEncoder()

In [114]:
our_heroes = [9,18,53]
their_heroes = [10,51,38]
hero_type = "Carry"
matchups = []
results = {}
check_heroes = remove_set(hero_ids,our_heroes + their_heroes)
checked_dict = {}
with open(hero_json_file) as heroes_file:
    heroes_json = json.load(heroes_file)
    for i in check_heroes:
        sum = 0
        if hero_type in next(item for item in heroes_json if item["id"] == i)['roles']:
            for j in their_heroes:
                if i != j:
                    result = model.predict_proba(ohe.transform([[str(i) + "-" + str(j)]]))[0].tolist()[0][0]
                    sum = sum + result
                    results[i] = sum

In [115]:
#If 3 or more heroes have the same maximum, get them all and get a random selection from them, otherwise show the best 3 hero choices
import random
max_value = max(results.values());  {key for key, value in results.items() if value == max_value}
if len({key for key, value in results.items() if value == max_value}) >= 3:
    possible_heroes = []
    print(len({key for key, value in results.items() if value == max_value}))
    for key,value in results.items():
        if value == max_value:
            possible_heroes.append(key)
    print(possible_heroes)
    hero_picks = random.sample(possible_heroes,3)
else:
    hero_picks = sorted(results, key=results.get, reverse=True)[:3]

49
[1, 4, 6, 8, 11, 15, 19, 21, 22, 23, 28, 32, 34, 35, 36, 39, 41, 43, 44, 46, 47, 48, 52, 56, 60, 61, 67, 70, 73, 75, 76, 77, 78, 80, 81, 82, 89, 93, 95, 99, 102, 106, 113, 114, 120, 126, 129, 135, 136]


In [116]:
print(hero_picks)

[136, 8, 19]
